**Forecast of power 3 hours ahead based on LSTM with knowledge of wind speed - with use of 7 days slices**

As a first step, an LSTM network is implemented considering only the power time series. The results can be compared with those of the AR(4) model fitted to the data in Matlab: 95% confidence intervals:

*   1h ahead: [-3.17:3.17]
*   2h ahead: [-4.73:4.73]
*   3h ahead: [-5.76:5.76]

And the LSTM network with slices:

*   1h ahead: [-3.12:3.12]
*   2h ahead: [-4.67:4.67]
*   3h ahead: [-5.72:5.72]



In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import statistics as stat
import random

import torch
import torch.optim as optim
import torch.nn as nn
from torch.autograd import Variable
from torch.nn.functional import relu, elu, relu6, sigmoid, tanh, softmax
from torch.nn.parameter import Parameter
import torch.nn.init as init
import torch.nn.functional as F

In [4]:
# Load the data and extract only the power time series and time
file = '\cex4WindDataInterpolated.csv'
data = pd.read_csv(file)
t = data.iloc[:,0].values
p = data.iloc[:,2].values
ws1 = data.iloc[:,3].values
ws2 = data.iloc[:,6].values
ws3 = data.iloc[:,9].values

FileNotFoundError: [Errno 2] No such file or directory: '\\cex4WindDataInterpolated.csv'

In [43]:
# Normalize the data except wind direction which will be one hot encoded
p_mean = np.nanmean(p)
p_std = np.nanstd(p)
ws_mean = np.nanmean(ws1)
ws_std = np.nanstd(ws1)

p = (p-p_mean)/p_std
ws1 = (ws1-ws_mean)/ws_std
ws2 = (ws2-ws_mean)/ws_std
ws3 = (ws3-ws_mean)/ws_std

In [44]:
# Loop to keep sequences of valid power and wind speed
p_valid, ws1_valid, ws2_valid, ws3_valid = [],[],[],[]
current_seq_p, current_seq_ws1, current_seq_ws2 , current_seq_ws3  = [],[],[],[]

nan = 1             # 1 if former value is nan

for i in range(len(p)):

  # Check if current value is nan
  if np.isnan(p[i]) or np.isnan(ws1[i]):
    # If former value not nan, store former sequence and create a new one 
    if nan == 0:
      p_valid.append(current_seq_p)
      ws1_valid.append(current_seq_ws1)
      ws2_valid.append(current_seq_ws2)
      ws3_valid.append(current_seq_ws3)
      current_seq_p, current_seq_ws1, current_seq_ws2 , current_seq_ws3  = [],[],[],[]
    nan = 1

  # Else, store the value and state it is not nan
  else:
    current_seq_p.append([p[i]])
    current_seq_ws1.append([ws1[i]])
    current_seq_ws2.append([ws2[i]])
    current_seq_ws3.append([ws3[i]])
    nan = 0

p_valid.append(current_seq_p)
ws1_valid.append(current_seq_ws1)
ws2_valid.append(current_seq_ws2)
ws3_valid.append(current_seq_ws3)

In [45]:
# Define slices of 168h power inputs and corresponding targets one 1h ahead
# Also define forecasts of wind speed
p_inputs, p_targets, p_targets2h, p_targets3h = [],[],[],[]
ws1_forecast, ws2_forecast, ws3_forecast = [],[],[]

for seq in range(len(p_valid)):
  for i in range(len(p_valid[seq])-170):
    p_inputs.append(p_valid[seq][i:i+168])
    p_targets.append(p_valid[seq][i+168])
    p_targets2h.append(p_valid[seq][i+169])
    p_targets3h.append(p_valid[seq][i+170])
    ws1_forecast.append(ws1_valid[seq][i+168])
    ws2_forecast.append(ws2_valid[seq][i+169])
    ws3_forecast.append(ws3_valid[seq][i+170])

In [46]:
# Shuffle the inputs and targets
random.seed(10)
l = list(zip(p_inputs,p_targets,p_targets2h,p_targets3h,ws1_forecast,ws2_forecast,ws3_forecast))
random.shuffle(l)
p_inputs,p_targets,p_targets2h,p_targets3h,ws1_forecast,ws2_forecast,ws3_forecast = zip(*l)

In [47]:
# Define an LSTM network

class MyRecurrentNet(nn.Module):
    def __init__(self):
        super(MyRecurrentNet, self).__init__()
        
        self.lstm = nn.LSTM(input_size =1,hidden_size=128, batch_first = False)        
        
        self.ffnn_forecast1 = nn.Linear(in_features = 1, out_features = 512, bias = False)
        self.batchnorm1 = nn.BatchNorm1d(512)
        self.ffnn_forecast2 = nn.Linear(in_features = 512, out_features = 512, bias = False)
        self.batchnorm2 = nn.BatchNorm1d(512)
        
        # Output layer
        self.l_out1 = nn.Linear(in_features=512+128,
                            out_features=64,
                            bias=False)
        self.batchnorm3 = nn.BatchNorm1d(64)
        self.l_out2 = nn.Linear(in_features=64,
                            out_features=1,
                            bias=False)
        
    def forward(self, p_past, ws_forecast):

        # RNN returns output
        x_rnn, (h, c) = self.lstm(p_past)
        
        # FNN on the wind speed forecast
        x_ffnn = self.ffnn_forecast1(ws_forecast)
        x_ffnn = self.batchnorm1(x_ffnn)
        x_ffnn = elu(x_ffnn)
        x_ffnn = self.ffnn_forecast2(x_ffnn)
        x_ffnn = self.batchnorm2(x_ffnn)
        x_ffnn = elu(x_ffnn)

        # Output layer on the concatenate last rnn hidden state and ffnn result
        x = torch.cat((x_rnn[-1], x_ffnn), 1)  # Concatenate on dimension 1, 0 being the batch samples, 1 being the units
        x = elu(self.l_out1(x))
        x = self.batchnorm3(x)
        x = self.l_out2(x)
        
        return x

net = MyRecurrentNet()
print(net)

MyRecurrentNet(
  (lstm): LSTM(1, 128)
  (ffnn_forecast1): Linear(in_features=1, out_features=512, bias=False)
  (batchnorm1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (ffnn_forecast2): Linear(in_features=512, out_features=512, bias=False)
  (batchnorm2): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (l_out1): Linear(in_features=640, out_features=64, bias=False)
  (batchnorm3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (l_out2): Linear(in_features=64, out_features=1, bias=False)
)


In [48]:
# Test of a dummy input of two sequences
dummy = torch.tensor(list([
                                [[1.0],[2.0],[3.0]],
                                [[4.0],[5.0],[6.0]]
                    ])).float()
dummy = torch.swapaxes(dummy,0,1) # The network uses the format (Seq, Batch, Features)
forecast = torch.tensor([[1.0],[2.0]]).float()
net(dummy,forecast)

tensor([[-0.0622],
        [ 0.0622]], grad_fn=<MmBackward>)

In [49]:
# Test the untrained network on two first sequences
net(torch.swapaxes(torch.Tensor([p_inputs[0],p_inputs[1]]),0,1),torch.Tensor([ws1_forecast[0],ws1_forecast[1]]))

tensor([[ 0.2925],
        [-0.2925]], grad_fn=<MmBackward>)

In [50]:
# Initialize the network
net = MyRecurrentNet()
# Convert to cuda if GPU available
if torch.cuda.is_available():
    print('##converting network to cuda-enabled')
    net.cuda()

# Define loss function and train parameters
criterion = nn.MSELoss(reduction='sum')    # Sum the squares but not perform the mean

# Adam gradient descent with learning rate decay
optimizer = optim.Adam(net.parameters(), lr=1e-3)
decayRate = 0.999
decay_scheduler = optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=decayRate)

# Length of the training and batches
epochs = 40
batch_size = 64
num_batch = len(p_inputs)//batch_size

# Function to get the batch
get_batch = lambda i, size: range(i * size, (i + 1) * size)

In [ ]:
# Training

# Track loss
training_RMSE = []

# Loop over epochs
for i in range(epochs):

    epoch_training_loss = 0

    net.train()

    # For each sequence in training set
    for b in range(num_batch):

      batch_index = get_batch(b,batch_size)
                 
      # Convert to tensor and swap axes of inputs to get (seq, batch, features)
      inputs_past = torch.swapaxes(torch.Tensor(p_inputs)[batch_index],0,1)
      inputs_forecast = torch.Tensor(ws1_forecast)[batch_index]
      targets = torch.Tensor(p_targets)[batch_index]

      # Convert to cuda to run on GPU
      if torch.cuda.is_available():
            inputs_past = Variable(inputs_past.cuda())
            inputs_forecast = Variable(inputs_forecast.cuda())
            targets = Variable(targets.cuda())

      # Forward pass
      outputs = net(inputs_past,inputs_forecast)
          
      # Compute loss
      loss = criterion(outputs, targets)
          
      # Backward pass
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
          
      # Update loss
      if torch.cuda.is_available():
        epoch_training_loss += loss.cpu().detach().numpy()
      else:
        epoch_training_loss += loss.detach().numpy()
    
    # Save loss for plot
    epoch_RMSE = np.sqrt(epoch_training_loss/(num_batch*batch_size))
    training_RMSE.append(epoch_RMSE*p_std) # Removing normalization to store RMSE and compute CI

    # Compute confidence interval
    CI = [norm.ppf(0.025)*training_RMSE[-1],norm.ppf(0.975)*training_RMSE[-1]]

    # Print loss every 10 epochs
    if i % 1 == 0:
        print('Epoch %d, training RMSE: %.2f, CI: [%.2f:%.2f]' % (i+1,training_RMSE[-1],CI[0],CI[1]))

    # Apply learning rate decay
    decay_scheduler.step()

# Plot training loss
epoch = np.arange(len(training_RMSE))
plt.figure()
plt.plot(epoch, training_RMSE, 'r')
plt.xlabel('Epoch'), plt.ylabel('Training RMSE')
plt.show()

Epoch 1, training RMSE: 1.96, CI: [-3.85:3.85]
Epoch 2, training RMSE: 1.79, CI: [-3.51:3.51]
Epoch 3, training RMSE: 1.78, CI: [-3.49:3.49]
Epoch 4, training RMSE: 1.76, CI: [-3.44:3.44]


In [ ]:
# Back on CPU
if torch.cuda.is_available():
  net.to('cpu')

In [ ]:
# Plot the learnt power curve for different past sequences
power = []
v = []
for i in range(40):
  v.append(i)
 # net(torch.swapaxes(torch.Tensor([p_inputs[0],p_inputs[1]]),0,1),torch.Tensor([ws1_forecast[0],ws1_forecast[1]]))
  power.append([net(torch.swapaxes(torch.Tensor([p_inputs[4]]),0,1),
                    torch.Tensor([[(i-ws_mean)/ws_std]])).detach().numpy(),
                net(torch.swapaxes(torch.Tensor([p_inputs[1]]),0,1),
                    torch.Tensor([[(i-ws_mean)/ws_std]])).detach().numpy(),
                net(torch.swapaxes(torch.Tensor([p_inputs[2]]),0,1),
                    torch.Tensor([[(i-ws_mean)/ws_std]])).detach().numpy(),
                net(torch.swapaxes(torch.Tensor([p_inputs[3]]),0,1),
                    torch.Tensor([[(i-ws_mean)/ws_std]])).detach().numpy(),
                net(torch.swapaxes(torch.Tensor([p_inputs[5]]),0,1),
                    torch.Tensor([[(i-ws_mean)/ws_std]])).detach().numpy()])

power = (np.squeeze(np.swapaxes(power,0,1),(2,3))*p_std)+p_mean

former_power = [p_inputs[4][-1],p_inputs[1][-1],p_inputs[2][-1],p_inputs[3][-1],p_inputs[5][-1]]

for i in range(5):
  plt.plot(v,power[i], label='$p_{n-1}$ = %.2f MW' %(p_mean+p_std*former_power[i][0]))
plt.xlabel('Wind speed [m/s]')
plt.ylabel('Power [MW]')
plt.legend()
plt.grid()

We see that independently of the wind speed, the network learnt from past sequences. If the former power was low, whatever high the forecasted wind speed is, the forecasted power will still be limited. This is expected.

In [ ]:
# Forecasting 1, 2 and 3 hours ahead

net.eval()

# Store predictions and errors
pred_1h = []
err_1h = []
pred_2h = []
err_2h = []
pred_3h = []
err_3h = []

# Loop over the sequences of valid data and ws forecast
for seq in range(len(p_inputs)-2):

    # Define past value for the 1h forecast
    past = p_inputs[seq]
    ws_forecast = ws1_forecast[seq]
    
    # Take output of first and only sequence, last value
    pred_1h.append(net(torch.swapaxes(torch.Tensor([past]),0,1),torch.Tensor([ws_forecast])).item())
    err_1h.append(pred_1h[-1]-p_targets[seq][0])

    # Repeat with prediction 2 hours ahead actualizing first the past values
    past.append([pred_1h[-1]])
    ws_forecast = ws2_forecast[seq]   #Take the forecast for 2h ahead made at least 2h before (ws2)
    pred_2h.append(net(torch.swapaxes(torch.Tensor([past]),0,1),torch.Tensor([ws_forecast])).item())
    err_2h.append(pred_2h[-1]-p_targets2h[seq][0])

    # Repeat with prediction 3 hours ahead
    past.append([pred_2h[-1]])
    ws_forecast = ws3_forecast[seq]
    pred_3h.append(net(torch.swapaxes(torch.Tensor([past]),0,1),torch.Tensor([ws_forecast])).item())
    err_3h.append(pred_3h[-1]-p_targets3h[seq][0])

    if seq % 1000 == 0:
      print('progress %.2f %%, RMSE 1h: %.2f, RMSE 2h: %.2f, RMSE 3h: %.2f'
           % (100*(seq+1)/(len(p_inputs)-2),
            p_std*np.sqrt(stat.mean(err_1h[n]**2 for n in range(len(err_1h)))),
            p_std*np.sqrt(stat.mean(err_2h[n]**2 for n in range(len(err_2h)))), 
            p_std*np.sqrt(stat.mean(err_3h[n]**2 for n in range(len(err_3h)))))
           )

In [ ]:
# Estimation of confidence intervals:
RMSE_1h = np.sqrt(stat.mean((p_std*err_1h[n])**2 for n in range(len(err_1h))))
RMSE_2h = np.sqrt(stat.mean((p_std*err_2h[n])**2 for n in range(len(err_2h))))
RMSE_3h = np.sqrt(stat.mean((p_std*err_3h[n])**2 for n in range(len(err_3h))))
CI_1h = [norm.ppf(0.025)*RMSE_1h,norm.ppf(0.975)*RMSE_1h]
CI_2h = [norm.ppf(0.025)*RMSE_2h,norm.ppf(0.975)*RMSE_2h]
CI_3h = [norm.ppf(0.025)*RMSE_3h,norm.ppf(0.975)*RMSE_3h]
print(f'Confidence interval 1h: {CI_1h}')
print(f'Confidence interval 2h: {CI_2h}')
print(f'Confidence interval 3h: {CI_3h}')

In [ ]:
# Save the model
torch.save(net.state_dict(),
           '/content/drive/MyDrive/Colab Notebooks/Time series/LSTM_Klim/LSTM_power_ws.pth')